In [1]:
import pandas as pd, numpy as np, json
from matches_loader import *
from members_loader import *
from clubs_loader import *
from point_utils import *

In [2]:
members=get_members('../data/manual/Evidenta membrilor.xlsm')

In [26]:
data={}
replace_active={'Activ':'Active','Inactiv':'Inactive','Free':'Inactive','AS':'Abroad',
                'Transferat':'Abroad','Decedat':'Inactive'}
active_redflags=['Deleted']
for i in members.T.iteritems():
    active=i[1][231]
    if active not in active_redflags:
        grades=i[1][['1 kyu','1 dan','2 dan','3 dan','4 dan','5 dan',
                     '6 dan','7 dan','8 dan']].replace('x',pd.NaT).dropna()
        grades0=i[1][['1 dan','2 dan','3 dan','4 dan','5 dan',
                      '6 dan','7 dan','8 dan']].replace('x',pd.NaT).dropna()
        df=pd.DataFrame(grades0)
        df.columns=['dan']
        df=df.reset_index().set_index('dan').sort_index()
        dummy={}
        grades=pd.to_datetime(grades.astype(str))
        active=replace_active[active]
        if len(grades)>0:
            mingrade=grades.min().year
            maxgrade=grades.max().year
        else:
            mingrade=2016 #default starting year
            maxgrade=2016
        if active=='Active':
            maxyear=2019 #default active max year
        else:
            maxyear=maxgrade+3 #default active years if unknown


        dummy['name']=i[1]['Numele']+' '+i[1]['Prenumele']
        dummy['birth']=str(i[1]['Data naşterii'])[:10]
        dummy['gen']=i[1]['Gen']
        dummy['ekf']=i[1]['Nr. EKF']
        dummy['active']=active
        club=i[1]['Club']
        dummy['transfer']=get_transfer(dummy['name'],i[1]['Unnamed: 3'])
         
        for year in range(mingrade,maxyear):

            #get year from exams
            dummy['dan']=len(df[:str
                                (year)])
            #get club from transfers
            clubs=clubs_loader.get_club_by_year(dummy['transfer'],club,year,mingrade,maxyear)
            clubs=clubs[:1] #remove this step to double count. this limits to first club in transfer years
            for j in range(len(clubs)):
                iclub=clubs_loader.replacer(clubs[j])
                dummy['club']=iclub
                dummy['age']=year-1-pd.to_datetime(dummy['birth']).year
                data=clubs_loader.add_to_club(data,iclub,year,dummy.copy())

all_data=[]

for club in data:
    for year in data[club]:
        df=pd.DataFrame(data[club][year])
        df['year']=year
        df['club']=club
        df=df.drop('transfer',axis=1)
        all_data.append(df)

members_clean=pd.concat(all_data).reset_index(drop=True)

In [30]:
members_clean.set_index('name').loc['Mureșan (Egri) Melinda']

,active,age,birth,club,dan,ekf,gen,year
name,,,,,,,,
Mureșan (Egri) Melinda,Inactive,14.0,1992-06-05,KYO,0,RO.00077,F,2007
Mureșan (Egri) Melinda,Inactive,15.0,1992-06-05,KYO,0,RO.00077,F,2008
Mureșan (Egri) Melinda,Inactive,16.0,1992-06-05,KYO,0,RO.00077,F,2009


In [28]:
members_clean

,active,age,birth,club,dan,ekf,gen,name,year
0,Inactive,27.0,1979-06-06,KYO,0,RO.00076,F,Abe (Carțiș) Emilia,2007
1,Active,36.0,1970-10-21,KYO,2,RO.00028,F,Grapă Daniela,2007
2,Inactive,34.0,1972-10-26,KYO,2,RO.00039,F,Hulea Violeta,2007
3,Active,25.0,1981-07-27,KYO,3,RO.00065,M,Józsa Levente,2007
4,Inactive,25.0,1981-10-18,KYO,0,RO.00079,M,Kovács Alexandru,2007
5,Inactive,14.0,1992-06-05,KYO,0,RO.00077,F,Mureșan (Egri) Melinda,2007
6,Abroad,30.0,1976-12-03,KYO,0,RO.00090,M,Ratz Bogdan,2007
7,Abroad,23.0,1983-09-29,KYO,0,RO.00071,F,Stoica Loredana,2007
8,Inactive,28.0,1979-06-06,KYO,0,RO.00076,F,Abe (Carțiș) Emilia,2008
9,Active,37.0,1970-10-21,KYO,3,RO.00028,F,Grapă Daniela,2008
